In [2]:
# Connet mssql to retrieve hotel data

import pymssql #引入pymssql模块
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import re



def conn():
    connect = pymssql.connect('121.42.61.195', 'test', 'tes@t123456#pwd2018&', 'Broker') #服务器名,账户,密码,数据库名
    if connect:
        print("连接成功!")
    return connect


conn = conn()

cursor = conn.cursor()   #创建一个游标对象,python里的sql语句都要通过cursor来执行
sql = "select planner_id, wedding_id, province, city, hotel, hotel_id from dbo.tb_hotel_wedding_planner_data"

df0 = pd.read_sql(sql,conn)
df=pd.DataFrame(df0)

print (df.head())

cursor.close()   
conn.close()

连接成功!
                             planner_id                            wedding_id  \
0  4d76d5e5-d9ee-4460-8bea-418a38dc4241  616c81a5-db1d-4a43-a930-ee993a2d15ff   
1  4d76d5e5-d9ee-4460-8bea-418a38dc4241  1d56c3f7-ad7e-4e2d-9118-ea3ba84a9e91   
2  4d76d5e5-d9ee-4460-8bea-418a38dc4241  9951f561-8623-4b32-919c-15362679f091   
3  4d76d5e5-d9ee-4460-8bea-418a38dc4241  ce9d2260-8ab5-419f-b54a-0eabcb383738   
4  4d76d5e5-d9ee-4460-8bea-418a38dc4241  127983ce-905a-4abb-b2e2-02a8b58ef81d   

  province  city             hotel hotel_id  
0      重庆市  None             北滨路仙厨     None  
1      重庆市  None          两江丽景科技馆店     None  
2      重庆市  None  找我婚礼茂业东方时代18楼37号     None  
3      重庆市  None             赛豪大酒店     None  
4      重庆市  None            爱情海俏巴渝     None  


In [3]:
# explore data, and then merge hotel name with province/city name into a new column
print ('unique wedding id: ', df.wedding_id.nunique(dropna=False))
print ('Null wedding id: ', df.wedding_id.isnull().sum())
print ('Non_Null wedding id: ', df.wedding_id.count()) # 非空元素个数
print ('dup wedding id: ', df.wedding_id.duplicated().sum())
#print ('dup wedding id: ', df.wedding_id.value_counts())
print (df.shape)

df['merged'] = df['province'].map(str) + ' '+ df['city'].map(str)+ ' ' +df['hotel'].map(str)
df['merged']  = df['merged'].map(lambda x: x.replace('None', ''))

line = "四川省  四川省尚锦翡翠酒店(成都春熙路店)府青路二段24号(近财富又一城,电子科大东校区)"
#line = "重庆市 重庆市 澳维酒店(西南门)"


# input: "四川省  四川省尚锦翡翠酒店(成都春熙路店)府青路二段24号(近财富又一城,电子科大东校区)"
# output: "四川省尚锦翡翠酒店(成都春熙路店)府青路二段24号(近财富又一城,电子科大东校区)"

def removeDupProvinceAndCity(line):
    if line:
        #print ("111", line)
        
        # remove dup province
        matchObj = re.search( r'(\b.+省\b)', line)
        if matchObj and len(matchObj.groups()) > 0:

            #print ("matchObj.group() :", matchObj)
            print ("province match, matchObj.group(1) : ", matchObj.group(1))
            result = line.find(matchObj.group(1), len(matchObj.group(1)))
            if result != -1:
                line = line[len(matchObj.group(1)):].strip()
                #print ("222", line)
        else:
            pass
            #print ("no match!")
            
        # remove dup city    
        matchObj = re.search( r'(\b.+?市\b)', line)
        if matchObj and len(matchObj.groups()) > 0:

            #print ("matchObj.group() :", matchObj)
            print ("city match, matchObj.group(1) : ", matchObj.group(1))
            result = line.find(matchObj.group(1), len(matchObj.group(1)))
            if result != -1:
                line = line[len(matchObj.group(1)):].strip()
                #print ("333", line)
        else:
            pass
            #print ("no match!")
    return line

#removeDupProvinceAndCity(line)
#print ("result ", line)

print (df['merged'].head(10))

df['merged']  = df['merged'].apply(lambda x: removeDupProvinceAndCity(x))
    
print (df['merged'].head(10))


# to save memory, only use first 1000 lines
#df = df[0:5000]


unique wedding id:  17199
Null wedding id:  0
Non_Null wedding id:  17245
dup wedding id:  46
(17245, 6)
0               重庆市  北滨路仙厨
1            重庆市  两江丽景科技馆店
2    重庆市  找我婚礼茂业东方时代18楼37号
3               重庆市  赛豪大酒店
4              重庆市  爱情海俏巴渝
5              重庆市  嘉州仙厨食府
6                  重庆市  未知
7           重庆市  渝中区文化宫阿兴记
8               重庆市  鱼洞陶然居
9               重庆市  曾家镇喜悦
Name: merged, dtype: object
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city 

city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
c

province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, matchObj.group(1) :  江西省
province match, 

province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, 

province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, 

...

province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, matchObj.group(1) :  四川省
province match, 

city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
c

city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
c

city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
c

city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
city match, matchObj.group(1) :  重庆市
c

In [4]:
import jieba


def word_segmentation(line):
    if line:
        #print (line)
        cut_words = list(jieba.cut(line))
        line = " ".join(cut_words)

    return line


df['merged_cut']  = df['merged'].apply(lambda x: word_segmentation(x))
   
print (df['merged'].tail())
print (df['merged_cut'].tail())
print (df.head())    


ModuleNotFoundError: No module named 'jieba'

In [4]:
def calcTfidfMatrix(document):
    tfidf_model = TfidfVectorizer().fit(document)
    #print(tfidf_model.vocabulary_)
    tfidf = tfidf_model.transform(document)

    simi_matrix = (tfidf * tfidf.T).A #similarities matrix of sentences
    simi_matrix = np.tril(simi_matrix) # only use the upper right part of the matrix
    return simi_matrix


In [6]:
# encoding=utf8  
import numpy as np

def save2excel(pd_simi_matrix):
    col_names = pd_simi_matrix.columns.values.tolist()
    row_names = pd_simi_matrix.index.values.tolist() 
    #print (row_names)
    threshold = 0.7

    pd_excel = None
    if os.path.exists("result.csv"):
        pd_excel = pd.read_csv("result.csv", index_col='ID')
        #print (pd_excel.tail())
        
    pd_simi_matrix = pd_simi_matrix.where(pd_simi_matrix > threshold, 0)

    for col in col_names:
        #rows_with_nlargest = pd.DataFrame(simi_matrix).nlargest(6, col) 
        rows_with_large_similarity = [row for row in row_names if pd_simi_matrix.loc[row, col] > 0]
        
        data = pd.DataFrame(df, columns=['planner_id', 'wedding_id', 'hotel', 'hotel_id', 'merged'], index=rows_with_large_similarity)
        
        data['recommended_name']= data['merged'].value_counts().idxmax()
        #print (data.head())
        #norepeat_df = data.drop_duplicates(subset=['recommended_name'], keep='first')
        #print (norepeat_df.head())
        if pd_excel is None:
            #data.to_csv("result.csv", index_label="ID")
            pd_excel = data
        else:
            #pd_tmp = pd.read_csv("result.csv", index_col='ID')
            #print ("111 ", pd_tmp.head())
            #pd_tmp = pd.concat([pd_tmp, data])
            #pd_tmp.to_csv("result.csv", index_label="ID")
            pd_excel = pd.concat([pd_excel, data])
            #print ("222 ", pd_excel.tail())
    
    # de-dup items with same index
    #idx = np.unique(pd_excel.index, return_index=True)[1]
    #pd_excel = pd_excel.iloc[idx]
    
    pd_excel['index_col'] = pd_excel.index
    pd_excel= pd_excel.drop_duplicates(subset=['index_col'], keep='first')
    #pd_excel.set_index = pd_excel['index']
    #pd_excel= pd_excel.drop('index', axis =1)
    
    pd_excel.to_csv("result.csv", index_label="ID")
    
    print ("end")

In [7]:
# split the big df to several small ones according to 'Province' columns

uni_values = df.province.unique()
print (uni_values) 
print ('Null  id: ', df.province.isnull().sum())
small_dfs = [df[df.province==i] for i in uni_values if i is not None]
print (len(small_dfs))
print (small_dfs[0].head())

batch_size = 3000
#steps = df.shape[0]//batch_size  # must be df's shape because small_df's index is from df

for sml_df in small_dfs:
    print (sml_df.province.head(2))
    steps = sml_df.shape[0]//batch_size
    for step in range(steps+1):
        if step == steps:
            batch_df = sml_df.iloc[step * batch_size:sml_df.shape[0]]
        else:
            batch_df = sml_df.iloc[step * batch_size:(step+1) * batch_size]
            
        simi_matrix = calcTfidfMatrix(batch_df['merged_cut'].tolist())
        pd_simi_matrix = pd.DataFrame(simi_matrix, columns=batch_df.index.values.tolist(), index=batch_df.index.values.tolist())
    
        save2excel(pd_simi_matrix)

stop

for step in range(steps+1):
    if step == steps:
        batch_df = df.iloc[step * batch_size:df.shape[0]]
    else:
        batch_df = df.iloc[step * batch_size:(step+1) * batch_size]
    #print (batch_df['merged'].head())
    simi_matrix = calcTfidfMatrix(batch_df['merged_cut'].tolist())
    pd_simi_matrix = pd.DataFrame(simi_matrix, columns=batch_df.index.values.tolist(), index=batch_df.index.values.tolist())
    #print (pd_simi_matrix.head())
    
    save2excel(pd_simi_matrix)

['重庆市' '贵州省' '海南省' '江西省' '四川省' '云南省' None]
Null  id:  67
6
                             planner_id                            wedding_id  \
0  fe8c2460-0971-4973-8a34-e42019e253d8  c20a17ed-98dc-45fb-b522-65d77753bc78   
1  fe8c2460-0971-4973-8a34-e42019e253d8  ca22e2ea-5963-4f14-887a-521ff372f0f0   
2  fe8c2460-0971-4973-8a34-e42019e253d8  78bf6ad4-dd9f-4a75-a4c9-191ad8f590fd   
3  fe8c2460-0971-4973-8a34-e42019e253d8  20b520d9-c55c-4bb4-ba41-7ef4fb180136   
4  fe8c2460-0971-4973-8a34-e42019e253d8  2bc66a9f-2d66-4b31-bb2a-d0301d1db85c   

  province city      hotel                              hotel_id  \
0      重庆市  重庆市  澳维酒店(西南门)  2632b834-759d-4e1b-a8d9-8ef4d7a40aec   
1      重庆市  重庆市   重庆凯宾斯基酒店  171b36dc-df60-4a0e-af60-34fb81114819   
2      重庆市  重庆市     佳宇英皇酒店  f05449d7-2a6c-4614-b77e-5c7fe8d5226f   
3      重庆市  重庆市   双山国际花园酒店  c25e0f1c-4fff-4ec6-bbe1-a2ecb558fd40   
4      重庆市  重庆市     银鑫世纪酒店  9ed4e2ad-857f-444b-8f32-75aaf4ecb882   

          merged            merged_cut  
0  重

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:35: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



end
end
end
end
end
end
end
end
end


NameError: name 'stop' is not defined